In [ ]:
!pip install librosa matplotlib numpy pillow
!pip uninstall pyarrow -y
!pip install --upgrade pyarrow datasets
!pip install datasets
!pip install evaluate
!pip install accelerate -U

In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import random

# Define paths for the dataset
path_to_data = '/kaggle/input/spectrograms-data/Spectrograms'
hc_folder = os.path.join(path_to_data, 'HC_AH')  # Healthy audio samples
pd_folder = os.path.join(path_to_data, 'PD_AH')  # Parkinson's audio samples

# Paths for saving spectrograms
spectrogram_hc_folder = os.path.join(path_to_data, 'healthy')
spectrogram_pd_folder = os.path.join(path_to_data, 'parkinson')

# Create directories if they do not exist
os.makedirs(spectrogram_hc_folder, exist_ok=True)
os.makedirs(spectrogram_pd_folder, exist_ok=True)

def create_spectrogram(audio_path, save_folder, file_name, chunk_size=0.05, sample_rate=22050, limit=0):
    y, sr = librosa.load(audio_path, sr=sample_rate)
    chunk_length = int(chunk_size * sr)
    total_chunks = len(y) // chunk_length
    if limit == 0:
        limit = total_chunks

    for i in range(limit):
        chunk = y[i * chunk_length: (i + 1) * chunk_length]
        S = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=64, n_fft=256, hop_length=64)
        S_DB = librosa.power_to_db(S, ref=np.max)

        plt.figure(figsize=(2, 2))
        plt.axis('off')
        librosa.display.specshow(S_DB, sr=sr, cmap='viridis')
        save_path = os.path.join(save_folder, f'{file_name}_chunk_{i}.png')
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
        plt.close()

def is_folder_empty(folder_path):
    return len(os.listdir(folder_path)) == 0

for folder, label, save_folder in zip([hc_folder, pd_folder], ['healthy', 'parkinson'], [spectrogram_hc_folder, spectrogram_pd_folder]):
    if is_folder_empty(save_folder):
        print(f"Generating spectrograms for {label} data...")
        for file in os.listdir(folder):
            if file.endswith('.wav'):
                file_path = os.path.join(folder, file)
                create_spectrogram(file_path, save_folder, file_name=os.path.splitext(file)[0])
    else:
        print(f"Spectrogram folder for {label} data already exists and is not empty. Skipping generation.")

def load_initial_images(spectrogram_folder, label):
    images = []
    labels = []
    for file in os.listdir(spectrogram_folder):
        if file.endswith('.png'):
            image_path = os.path.join(spectrogram_folder, file)
            img = Image.open(image_path).convert('RGB')
            img = img.resize((224, 224))
            images.append(np.array(img))
            labels.append(label)
    return images, labels

healthy_images, healthy_labels = load_initial_images(spectrogram_hc_folder, 'healthy')
parkinson_images, parkinson_labels = load_initial_images(spectrogram_pd_folder, 'parkinson')

target_count = min(len(healthy_images), len(parkinson_images))

def load_dataset_with_limit(audio_folder, spectrogram_folder, label, target_count):
    images, labels = load_initial_images(spectrogram_folder, label)
    selected_indices = random.sample(range(len(images)), target_count)
    images = [images[i] for i in selected_indices]
    labels = [labels[i] for i in selected_indices]
    return images, labels

print("Count : " + str(target_count))

healthy_images, healthy_labels = load_dataset_with_limit(hc_folder, spectrogram_hc_folder, 'healthy', target_count)
print(len(healthy_images))
parkinson_images, parkinson_labels = load_dataset_with_limit(pd_folder, spectrogram_pd_folder, 'parkinson', target_count)
print(len(parkinson_images))

images = healthy_images + parkinson_images
labels = healthy_labels + parkinson_labels

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

train_data = {"image": train_images, "label": train_labels}
test_data = {"image": test_images, "label": test_labels}

train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': test_dataset
})

print(dataset)

In [ ]:
import requests
import torch
from PIL import Image
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from tqdm import tqdm
from evaluate import load
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import accelerate
import transformers

device = "cpu"

model_name = "google/vit-base-patch16-224"                        # the model name
image_processor = ViTImageProcessor.from_pretrained(model_name)   # load the image processor
modelViT = ViTForImageClassification.from_pretrained(model_name)     # loading the pre-trained model

labels = ['healthy', 'parkinson']

label_map = {'healthy': 0, 'parkinson': 1}

In [ ]:
def collate_fn(batch):
    # Split the batch into images (X) and labels (y)
    images = [item['image'] for item in batch]  # First element in each item is the image
    labels = [item['label'] for item in batch]  # Second element in each item is the label

    # Convert images to tensors
    images_tensor = torch.stack([torch.tensor(img, dtype=torch.float32).permute(2, 0, 1) for img in images])  # Convert images to tensors and change dimension order

    # Convert labels from strings ('healthy', 'parkinson') to integers (0, 1)
    labels_tensor = torch.tensor([label_map[label] for label in labels], dtype=torch.long)

    return {
        'pixel_values': images_tensor,  # Image tensor
        'labels': labels_tensor         # Label tensor (numerical)
    }

In [ ]:
# Load the accuracy and f1 metrics from the evaluate module
accuracy = load("accuracy")
f1 = load("f1")

def compute_metrics(eval_pred):
    # Predictions and true labels
    predictions = eval_pred.predictions
    references = eval_pred.label_ids

    # Compute accuracy and F1 scores
    accuracy_score = accuracy.compute(predictions=np.argmax(predictions, axis=1), references=references)
    f1_score = f1.compute(predictions=np.argmax(predictions, axis=1), references=references, average="macro")

    # Compute AUC (for multiclass classification, use 'ovo' or 'ovr')
    try:
        auc_score = roc_auc_score(references, predictions, multi_class="ovr", average="macro")
    except ValueError:
        # Handle cases where AUC cannot be computed (e.g., single-class predictions)
        auc_score = np.nan

    # Plot the ROC curve
    plt.figure(figsize=(8, 6))
    if len(predictions.shape) > 1 and predictions.shape[1] > 1:
        # Multiclass ROC curve
        for i in range(predictions.shape[1]):
            fpr, tpr, _ = roc_curve(references == i, predictions[:, i])
            plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc_score(references == i, predictions[:, i]):.2f})")
    else:
        # Binary ROC curve
        fpr, tpr, _ = roc_curve(references, predictions[:, 1])
        plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")

    # Configure plot
    plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

    # Return all metrics
    return {**accuracy_score, **f1_score, "auc": auc_score}

In [ ]:
# load the ViT model
modelViT = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True,
)

# defining the training arguments
training_args = TrainingArguments(
  output_dir="/kaggle/working/vit-base", # output directory
  per_device_train_batch_size=32, # batch size per device during training
  eval_strategy="steps",    # evaluation strategy to adopt during training
  num_train_epochs=20,             # total number of training epochs
  # fp16=True,                    # use mixed precision
  save_steps=80,                # number of update steps before saving checkpoint
  eval_steps=80,                # number of update steps before evaluating
  logging_steps=80,             # number of update steps before logging
  save_total_limit=2,             # limit the total amount of checkpoints on disk
  remove_unused_columns=False,    # remove unused columns from the dataset
  push_to_hub=False,              # do not push the model to the hub
  report_to='tensorboard',        # report metrics to tensorboard
  load_best_model_at_end=True,    # load the best model at the end of training
)

In [ ]:
trainerViT = Trainer(
    model=modelViT,                        # the instantiated Transformers model to be trained
    args=training_args,                 # training arguments, defined above
    data_collator=collate_fn,           # the data collator that will be used for batching
    compute_metrics=compute_metrics,    # the metrics function that will be used for evaluation
    train_dataset=dataset["train"],     # training dataset
    eval_dataset=dataset["validation"], # evaluation dataset
    tokenizer=image_processor,          # the processor that will be used for preprocessing the images
)

trainerViT.train()

torch.save(modelViT.state_dict(), '/kaggle/working/ViT_model.pth')

In [ ]:
# ResNet
from transformers import AutoFeatureExtractor, ResNetForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from evaluate import load
import requests
import torch
from PIL import Image
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from tqdm import tqdm
from evaluate import load
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import accelerate
import transformers

In [ ]:
labels = ['healthy', 'parkinson']

label_map = {'healthy': 0, 'parkinson': 1}

def collate_fn(batch):
    # Split the batch into images (X) and labels (y)
    images = [item['image'] for item in batch]  # First element in each item is the image
    labels = [item['label'] for item in batch]  # Second element in each item is the label

    # Convert images to tensors
    images_tensor = torch.stack([torch.tensor(img, dtype=torch.float32).permute(2, 0, 1) for img in images])  # Convert images to tensors and change dimension order

    # Convert labels from strings ('healthy', 'parkinson') to integers (0, 1)
    labels_tensor = torch.tensor([label_map[label] for label in labels], dtype=torch.long)

    return {
        'pixel_values': images_tensor,  # Image tensor
        'labels': labels_tensor         # Label tensor (numerical)
    }

In [ ]:
# Load the accuracy and f1 metrics from the evaluate module
accuracy = load("accuracy")
f1 = load("f1")

def compute_metrics(eval_pred):
    # Predictions and true labels
    predictions = eval_pred.predictions
    references = eval_pred.label_ids

    # Compute accuracy and F1 scores
    accuracy_score = accuracy.compute(predictions=np.argmax(predictions, axis=1), references=references)
    f1_score = f1.compute(predictions=np.argmax(predictions, axis=1), references=references, average="macro")

    # Compute AUC (for multiclass classification, use 'ovo' or 'ovr')
    try:
        auc_score = roc_auc_score(references, predictions, multi_class="ovr", average="macro")
    except ValueError:
        # Handle cases where AUC cannot be computed (e.g., single-class predictions)
        auc_score = np.nan

    # Plot the ROC curve
    plt.figure(figsize=(8, 6))
    if len(predictions.shape) > 1 and predictions.shape[1] > 1:
        # Multiclass ROC curve
        for i in range(predictions.shape[1]):
            fpr, tpr, _ = roc_curve(references == i, predictions[:, i])
            plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc_score(references == i, predictions[:, i]):.2f})")
    else:
            # Binary ROC curve
        fpr, tpr, _ = roc_curve(references, predictions[:, 1])
        plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")

    # Configure plot
    plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

    # Return all metrics
    return {**accuracy_score, **f1_score, "auc": auc_score}

In [ ]:
modelResNet = ResNetForImageClassification.from_pretrained(
    'microsoft/resnet-50',
    num_labels=2,  # Binary classification
    ignore_mismatched_sizes=True
)

# training arguments
training_args = TrainingArguments(
    output_dir="./results", # output directory
  per_device_train_batch_size=32, # batch size per device during training
  eval_strategy="steps",    # evaluation strategy to adopt during training
  num_train_epochs=20,             # total number of training epochs
  save_steps=80,                # number of update steps before saving checkpoint
  eval_steps=80,                # number of update steps before evaluating
  logging_steps=80,             # number of update steps before logging
  save_total_limit=2,             # limit the total amount of checkpoints on disk
  remove_unused_columns=False,    # remove unused columns from the dataset
  push_to_hub=False,
report_to='tensorboard',        # report metrics to tensorboard
  load_best_model_at_end=True,
)

In [ ]:
# Loading a pre-trained ResNet feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained('microsoft/resnet-50')

# Initializing the Trainer
trainerResNet = Trainer(
    model=modelResNet,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=feature_extractor,  # Use the feature extractor here
    compute_metrics=compute_metrics
)

trainerResNet.train()

torch.save(modelResNet.state_dict(), '/kaggle/working/ResNet_model.pth')

In [ ]:
import torch

labels = ['healthy', 'parkinson']

label_map = {'healthy': 0, 'parkinson': 1}

def collate_fn(batch):
    # Split the batch into images (X) and labels (y)
    # print(batch)
    images = [item['image'] for item in batch]  # First element in each item is the image
    labels = [item['label'] for item in batch]  # Second element in each item is the label

    # Convert images to tensors
    images_tensor = torch.stack([torch.tensor(img, dtype=torch.float32).permute(2, 0, 1) for img in images])  # Convert images to tensors and change dimension order

    # Convert labels from strings ('healthy', 'parkinson') to integers (0, 1)
    labels_tensor = torch.tensor([label_map[label] for label in labels], dtype=torch.long)

    return {
        'pixel_values': images_tensor,  # Image tensor
        'labels': labels_tensor         # Label tensor (numerical)
    }

In [ ]:
from transformers import AutoFeatureExtractor, MobileNetV2ForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import torch
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torchvision.transforms import Compose, Normalize, Resize, ToTensor

In [ ]:
# Metrics for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary', zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Load a pre-trained MobileNetV2 model
model = MobileNetV2ForImageClassification.from_pretrained(
    "google/mobilenet_v2_1.0_224",
    num_labels=2,
    ignore_mismatched_sizes=True
)

# Load a pre-trained feature extractor for MobileNetV2
feature_extractor = AutoFeatureExtractor.from_pretrained("google/mobilenet_v2_1.0_224")

# Define transformations for images
transform = Compose([
    Resize((224, 224), antialias=True),
    ToTensor(),
    Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

def preprocess(example):
    """Process a single example."""
    # Get the image from the example - using 'image' instead of 'pixel_values'
    image = example['image']

    # Convert to numpy array if needed
    if isinstance(image, list):
        image = np.array(image, dtype=np.uint8)

    # Handle different image formats
    if len(image.shape) == 4:
        image = np.squeeze(image, axis=(0, 1))

    if len(image.shape) == 2:  # Grayscale
        image = np.stack([image] * 3, axis=-1)
    elif len(image.shape) == 3 and image.shape[0] == 3:  # Channels first
        image = np.transpose(image, (1, 2, 0))

    # Convert to PIL and transform
    image = Image.fromarray(image)
    pixel_values = transform(image)

    label_mapping = {'healthy': 0, 'parkinson': 1}
    label = label_mapping[example['label']]

    # Return the processed example
    return {
        'pixel_values': pixel_values,
        'label': label
    }

# Apply preprocessing to dataset
processed_dataset = dataset.map(
    preprocess,
    remove_columns=['image'],  # Remove only the 'image' column
    num_proc=4,
    desc="Processing images"
)

In [ ]:
# Data collator
def collate_fn(batch):
    # Convert image lists and labels into tensors directly
    pixel_values = torch.tensor([np.array(example["pixel_values"]) for example in batch], dtype=torch.float32)
    labels = torch.tensor([example["label"] for example in batch], dtype=torch.long)

    return {
        "pixel_values": pixel_values,
        "labels": labels
    }

# Test the collation
test_batch = [processed_dataset['train'][i] for i in range(4)]
test_collated = collate_fn(test_batch)
print("Collated batch shapes:",
      "\nPixel values:", test_collated["pixel_values"].shape,
      "\nLabels:", test_collated["labels"].shape)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_steps=80,
    save_steps=80,
    logging_steps=80,
    num_train_epochs=20,
    eval_strategy="steps",
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
    fp16=True,
    gradient_accumulation_steps=2,
    warmup_steps=500,
    weight_decay=0.01,
    metric_for_best_model="f1",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/MobileNet_model.pth')

In [ ]:
from transformers import ViTForImageClassification
import torch

labels = ['healthy', 'parkinson']

# Re-initialize the model architecture with correct label mappings
modelViT_loaded = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True,
)

# Load the saved weights
modelViT_loaded.load_state_dict(torch.load("/kaggle/input/vit_parkinson/pytorch/default/1/ViT_model.pth", map_location=torch.device("cpu")))

In [ ]:
from torchvision import transforms
from transformers import AutoFeatureExtractor

# Load a pre-trained feature extractor for MobileNetV2
feature_extractor = AutoFeatureExtractor.from_pretrained("google/mobilenet_v2_1.0_224")

transform = transforms.Compose([
    transforms.Resize((224, 224), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

In [ ]:
from transformers import ResNetForImageClassification

# Recreate the model architecture
modelResNet_loaded = ResNetForImageClassification.from_pretrained(
    'microsoft/resnet-50',
    num_labels=2,
    ignore_mismatched_sizes=True,
)

# Load the saved weights into the model
modelResNet_loaded.load_state_dict(torch.load("/kaggle/input/resnet_parkinson/pytorch/default/1/ResNet_model.pth", map_location=torch.device("cpu")))

In [ ]:
from transformers import MobileNetV2ForImageClassification

# Re-initialize the model architecture
modelMobileNet_loaded = MobileNetV2ForImageClassification.from_pretrained(
    "google/mobilenet_v2_1.0_224",
    num_labels=2,
    ignore_mismatched_sizes=True
)

# Load the state_dict
modelMobileNet_loaded.load_state_dict(torch.load("/kaggle/input/mobilenet_parkinson/pytorch/default/1/MobileNet_model.pth", map_location=torch.device("cpu")))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

modelResNet_loaded = modelResNet_loaded.to(device)
modelViT_loaded = modelViT_loaded.to(device)
modelMobileNet_loaded = modelMobileNet_loaded.to(device)

basic_transform = transforms.ToTensor()

In [ ]:
def predict_ensemble(image):
    x1 = transform(image).unsqueeze(0).to(device)
    x = basic_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        r = modelResNet_loaded(x)
        v = modelViT_loaded(x)
        m = modelMobileNet_loaded(x1)

    # Average predictions
    avg_logits = (r.logits + v.logits + m.logits) / 3
    pred = torch.argmax(avg_logits, dim=1).item()

    return pred

In [ ]:
from tqdm import tqdm

predictions = []
true_labels = []

for example in tqdm(dataset["validation"]):
    raw_image = example['image']

    # Ensure image is in HWC format for PIL conversion
    if isinstance(raw_image, list):
        raw_image = np.array(raw_image, dtype=np.uint8)

    if isinstance(raw_image, torch.Tensor):
        raw_image = raw_image.permute(1, 2, 0).numpy()  # CHW → HWC

    if len(raw_image.shape) == 2:  # Grayscale
        raw_image = np.stack([raw_image] * 3, axis=-1)
    elif len(raw_image.shape) == 3 and raw_image.shape[0] == 3:  # CHW
        raw_image = np.transpose(raw_image, (1, 2, 0))  # Convert to HWC

    pil_image = Image.fromarray(raw_image.astype(np.uint8))

    # Don't transform here. Let predict_ensemble handle it.
    pred = predict_ensemble(pil_image)

    predictions.append(pred)
    if example['label'] == "healthy":
        true_labels.append(0)
    else:
        true_labels.append(1)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
print(true_labels[0])
print(predictions[0])
print(classification_report(true_labels, predictions, target_names=["healthy", "parkinson"]))
print("Accuracy:", accuracy_score(true_labels, predictions))
print("F1 Score:",f1_score(true_labels,predictions))

In [ ]:
def predict_ensemble_hardvote(image):
    x1 = transform(image).unsqueeze(0).to(device)
    x = basic_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        # Get predicted class from each model (0 or 1)
        r_pred = torch.argmax(modelResNet_loaded(x).logits, dim=1).item()
        v_pred = torch.argmax(modelViT_loaded(x).logits, dim=1).item()
        m_pred = torch.argmax(modelMobileNet_loaded(x1).logits, dim=1).item()

    # Majority voting
    preds = [r_pred, v_pred, m_pred]
    final_pred = round(sum(preds) / len(preds))  # returns 1 if 2 or more are 1, else 0

    return final_pred

In [ ]:
from tqdm import tqdm

predictions = []
true_labels = []

for example in tqdm(dataset["validation"]):
    raw_image = example['image']

    # Ensure image is in HWC format for PIL conversion
    if isinstance(raw_image, list):
        raw_image = np.array(raw_image, dtype=np.uint8)

    if isinstance(raw_image, torch.Tensor):
        raw_image = raw_image.permute(1, 2, 0).numpy()  # CHW → HWC

    if len(raw_image.shape) == 2:  # Grayscale
        raw_image = np.stack([raw_image] * 3, axis=-1)
    elif len(raw_image.shape) == 3 and raw_image.shape[0] == 3:  # CHW
        raw_image = np.transpose(raw_image, (1, 2, 0))  # Convert to HWC

    pil_image = Image.fromarray(raw_image.astype(np.uint8))

    # Don't transform here. Let predict_ensemble handle it.
    pred = predict_ensemble_hardvote(pil_image)

    predictions.append(pred)
    if example['label'] == "healthy":
        true_labels.append(0)
    else:
        true_labels.append(1)

In [ ]:
# testing using hardVoting
from sklearn.metrics import classification_report, accuracy_score, f1_score
print(true_labels[0])
print(predictions[0])
print(classification_report(true_labels, predictions, target_names=["healthy", "parkinson"]))
print("Accuracy:", accuracy_score(true_labels, predictions))
print("F1 Score:",f1_score(true_labels,predictions))